# Análisis de Negocio - Formula 1 Machine Learning

## 1. Introducción

Este notebook presenta el entendimiento del negocio y análisis exploratorio para el proyecto de predicción en Formula 1. Se identificarán dos casos de uso principales:

1. **Caso de Regresión**: Predicción del tiempo de vuelta promedio de un piloto en una carrera
2. **Caso de Clasificación**: Predicción de si un piloto terminará en el podio (Top 3)

### 1.1 Objetivos del Análisis

- Comprender la estructura y relaciones entre los datasets de Formula 1
- Identificar variables clave para predicciones
- Definir métricas de éxito y valor de negocio
- Establecer un plan de implementación claro

## 2. Configuración del Entorno

In [9]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## 3. Carga y Exploración de Datos

### 3.1 Carga de Datasets

In [10]:
# Definición de rutas de datos
DATA_PATH = '../data/01_raw/'

# Diccionario para almacenar todos los datasets
datasets = {}

# Lista de archivos a cargar
files_to_load = [
    'Driver_Details.csv',
    'Driver_Rankings.csv',
    'Constructor_Performance.csv',
    'Constructor_Rankings.csv',
    'Race_Results.csv',
    'Qualifying_Results.csv',
    'Lap_Timings.csv',
    'Pit_Stop_Records.csv',
    'Race_Schedule.csv',
    'Race_Status.csv',
    'Season_Summaries.csv',
    'Sprint_Race_Results.csv',
    'Team_Details.csv',
    'Track_Information.csv'
]

# Carga de todos los datasets
print("Cargando datasets de Formula 1...\n")
for file in files_to_load:
    dataset_name = file.replace('.csv', '').lower()
    try:
        datasets[dataset_name] = pd.read_csv(DATA_PATH + file)
        print(f"✓ {file} cargado exitosamente")
    except Exception as e:
        print(f"✗ Error cargando {file}: {e}")

print(f"\nTotal de datasets cargados: {len(datasets)}")

Cargando datasets de Formula 1...

✓ Driver_Details.csv cargado exitosamente
✓ Driver_Rankings.csv cargado exitosamente
✓ Constructor_Performance.csv cargado exitosamente
✓ Constructor_Rankings.csv cargado exitosamente
✓ Race_Results.csv cargado exitosamente
✓ Qualifying_Results.csv cargado exitosamente
✓ Lap_Timings.csv cargado exitosamente
✓ Pit_Stop_Records.csv cargado exitosamente
✓ Race_Schedule.csv cargado exitosamente
✓ Race_Status.csv cargado exitosamente
✓ Season_Summaries.csv cargado exitosamente
✓ Sprint_Race_Results.csv cargado exitosamente
✓ Team_Details.csv cargado exitosamente
✓ Track_Information.csv cargado exitosamente

Total de datasets cargados: 14


### 3.2 Descripción de Tablas

Análisis detallado de cada tabla y su propósito en el contexto de Formula 1:

In [11]:
# Descripción de cada dataset
dataset_descriptions = {
    'driver_details': 'Información personal y demográfica de pilotos (nombre, nacionalidad, fecha de nacimiento)',
    'driver_rankings': 'Rankings históricos de pilotos por temporada y carrera',
    'constructor_performance': 'Rendimiento de constructores/equipos por carrera',
    'constructor_rankings': 'Rankings de constructores en el campeonato',
    'race_results': 'Resultados detallados de cada carrera (posiciones, puntos, tiempos)',
    'qualifying_results': 'Resultados de sesiones de clasificación (Q1, Q2, Q3)',
    'lap_timings': 'Tiempos de vuelta individuales durante las carreras',
    'pit_stop_records': 'Registros de paradas en boxes (duración, vuelta)',
    'race_schedule': 'Calendario de carreras con fechas y ubicaciones',
    'race_status': 'Estados finales de pilotos en carreras (finalizado, retirado, etc.)',
    'season_summaries': 'Resúmenes estadísticos por temporada',
    'sprint_race_results': 'Resultados de carreras sprint (formato especial)',
    'team_details': 'Información de equipos/constructores',
    'track_information': 'Características de circuitos (ubicación, longitud, país)'
}

print("DESCRIPCIÓN DE DATASETS\n" + "="*80)
for name, description in dataset_descriptions.items():
    if name in datasets:
        df = datasets[name]
        print(f"\n📊 {name.upper().replace('_', ' ')}")
        print(f"   Descripción: {description}")
        print(f"   Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
        print(f"   Columnas: {', '.join(df.columns[:8])}{'...' if len(df.columns) > 8 else ''}")

DESCRIPCIÓN DE DATASETS

📊 DRIVER DETAILS
   Descripción: Información personal y demográfica de pilotos (nombre, nacionalidad, fecha de nacimiento)
   Dimensiones: 859 filas × 9 columnas
   Columnas: driverId, driverRef, number, code, forename, surname, dob, nationality...

📊 DRIVER RANKINGS
   Descripción: Rankings históricos de pilotos por temporada y carrera
   Dimensiones: 34,595 filas × 7 columnas
   Columnas: driverStandingsId, raceId, driverId, points, position, positionText, wins

📊 CONSTRUCTOR PERFORMANCE
   Descripción: Rendimiento de constructores/equipos por carrera
   Dimensiones: 12,505 filas × 5 columnas
   Columnas: constructorResultsId, raceId, constructorId, points, status

📊 CONSTRUCTOR RANKINGS
   Descripción: Rankings de constructores en el campeonato
   Dimensiones: 13,271 filas × 7 columnas
   Columnas: constructorStandingsId, raceId, constructorId, points, position, positionText, wins

📊 RACE RESULTS
   Descripción: Resultados detallados de cada carrera (posicio

### 3.3 Análisis de Estructura de Datos

In [12]:
# Análisis detallado de cada dataset
def analyze_dataset(df, name):
    """Análisis completo de un dataset"""
    print(f"\n{'='*80}")
    print(f"ANÁLISIS DE: {name.upper().replace('_', ' ')}")
    print(f"{'='*80}")
    
    # Información básica
    print(f"\n📋 Información General:")
    print(f"   - Total de registros: {len(df):,}")
    print(f"   - Total de columnas: {len(df.columns)}")
    print(f"   - Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Tipos de datos
    print(f"\n📊 Tipos de Datos:")
    dtype_counts = df.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"   - {dtype}: {count} columnas")
    
    # Valores nulos
    null_counts = df.isnull().sum()
    null_cols = null_counts[null_counts > 0]
    
    if len(null_cols) > 0:
        print(f"\n⚠️ Columnas con Valores Nulos:")
        for col, count in null_cols.items():
            percentage = (count / len(df)) * 100
            print(f"   - {col}: {count:,} ({percentage:.1f}%)")
    else:
        print(f"\n✓ No hay valores nulos en este dataset")
    
    # Columnas clave identificadas
    print(f"\n🔑 Columnas del Dataset:")
    for i, col in enumerate(df.columns, 1):
        dtype_str = str(df[col].dtype)
        print(f"   {i:2}. {col:30} ({dtype_str})")
    
    # Estadísticas para columnas numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\n📈 Resumen Estadístico (columnas numéricas):")
        print(df[numeric_cols].describe().round(2).to_string())
    
    return df.info()

# Analizar datasets principales
key_datasets = ['race_results', 'driver_details', 'lap_timings', 'qualifying_results']
for dataset_name in key_datasets:
    if dataset_name in datasets:
        analyze_dataset(datasets[dataset_name], dataset_name)


ANÁLISIS DE: RACE RESULTS

📋 Información General:
   - Total de registros: 26,519
   - Total de columnas: 18
   - Memoria utilizada: 13.55 MB

📊 Tipos de Datos:
   - object: 9 columnas
   - int64: 8 columnas
   - float64: 1 columnas

✓ No hay valores nulos en este dataset

🔑 Columnas del Dataset:
    1. resultId                       (int64)
    2. raceId                         (int64)
    3. driverId                       (int64)
    4. constructorId                  (int64)
    5. number                         (object)
    6. grid                           (int64)
    7. position                       (object)
    8. positionText                   (object)
    9. positionOrder                  (int64)
   10. points                         (float64)
   11. laps                           (int64)
   12. time                           (object)
   13. milliseconds                   (object)
   14. fastestLap                     (object)
   15. rank                           (object)
  

## 4. Entendimiento del Negocio

### 4.1 Contexto de Formula 1

La Formula 1 representa el pináculo del automovilismo mundial, donde la combinación de:
- **Tecnología**: Desarrollo aereodinámico, motores híbridos, telemetría avanzada
- **Estrategia**: Gestión de neumáticos, paradas en boxes, consumo de combustible
- **Talento humano**: Habilidad de pilotos, decisiones de ingenieros

determina el éxito en cada carrera.

### 4.2 Stakeholders y sus Necesidades

In [13]:
# Definición de stakeholders y casos de uso
stakeholders = pd.DataFrame({
    'Stakeholder': [
        'Equipos de F1',
        'Pilotos',
        'Patrocinadores',
        'Medios de Comunicación',
        'Aficionados',
        'Casas de Apuestas',
        'FIA/Organizadores'
    ],
    'Necesidad Principal': [
        'Optimización de estrategia de carrera',
        'Análisis de rendimiento personal',
        'ROI de inversión y visibilidad',
        'Contenido analítico y predicciones',
        'Insights y estadísticas interesantes',
        'Modelos de probabilidad precisos',
        'Análisis de competitividad y seguridad'
    ],
    'Valor del ML': [
        'Decisiones estratégicas basadas en datos',
        'Identificación de áreas de mejora',
        'Predicción de exposición mediática',
        'Generación automática de contenido',
        'Experiencia enriquecida',
        'Mejora en precisión de cuotas',
        'Detección de patrones y anomalías'
    ]
})

print("ANÁLISIS DE STAKEHOLDERS")
print("="*100)
print(stakeholders.to_string(index=False))

ANÁLISIS DE STAKEHOLDERS
           Stakeholder                    Necesidad Principal                             Valor del ML
         Equipos de F1  Optimización de estrategia de carrera Decisiones estratégicas basadas en datos
               Pilotos       Análisis de rendimiento personal        Identificación de áreas de mejora
        Patrocinadores         ROI de inversión y visibilidad       Predicción de exposición mediática
Medios de Comunicación     Contenido analítico y predicciones       Generación automática de contenido
           Aficionados   Insights y estadísticas interesantes                  Experiencia enriquecida
     Casas de Apuestas       Modelos de probabilidad precisos            Mejora en precisión de cuotas
     FIA/Organizadores Análisis de competitividad y seguridad        Detección de patrones y anomalías


## 5. Definición de Casos de Uso de Machine Learning

### 5.1 Caso de Regresión: Predicción de Tiempo de Vuelta Promedio

In [14]:
# Exploración para caso de regresión
print("CASO DE USO 1: REGRESIÓN - Predicción de Tiempo de Vuelta Promedio")
print("="*80)

# Análisis de tiempos de vuelta
if 'lap_timings' in datasets:
    lap_data = datasets['lap_timings']
    
    print("\n📊 Análisis de Tiempos de Vuelta:")
    print(f"   - Total de registros de vueltas: {len(lap_data):,}")
    
    # Verificar columnas disponibles
    if 'LapTime' in lap_data.columns:
        # Estadísticas de tiempos
        lap_stats = lap_data['LapTime'].describe()
        print(f"\n   Estadísticas de Tiempos de Vuelta:")
        print(f"   - Tiempo promedio: {lap_stats['mean']:.2f} segundos")
        print(f"   - Desviación estándar: {lap_stats['std']:.2f} segundos")
        print(f"   - Tiempo mínimo: {lap_stats['min']:.2f} segundos")
        print(f"   - Tiempo máximo: {lap_stats['max']:.2f} segundos")

# Definición del problema de regresión
regression_problem = pd.DataFrame({
    'Aspecto': [
        'Variable Objetivo',
        'Variables Predictoras',
        'Métrica Principal',
        'Métricas Secundarias',
        'Valor de Negocio',
        'Desafíos Técnicos'
    ],
    'Descripción': [
        'Tiempo promedio de vuelta en una carrera específica',
        'Posición de salida, tiempos de clasificación, histórico del piloto, características del circuito, condiciones climáticas',
        'Mean Absolute Error (MAE) en segundos',
        'RMSE, R², MAPE',
        'Optimización de estrategia de neumáticos y combustible durante la carrera',
        'Variabilidad por condiciones climáticas, incidentes en pista, safety cars'
    ]
})

print("\n📋 Definición del Problema de Regresión:")
for _, row in regression_problem.iterrows():
    print(f"\n   {row['Aspecto']}:")
    print(f"   → {row['Descripción']}")

CASO DE USO 1: REGRESIÓN - Predicción de Tiempo de Vuelta Promedio

📊 Análisis de Tiempos de Vuelta:
   - Total de registros de vueltas: 575,029

📋 Definición del Problema de Regresión:

   Variable Objetivo:
   → Tiempo promedio de vuelta en una carrera específica

   Variables Predictoras:
   → Posición de salida, tiempos de clasificación, histórico del piloto, características del circuito, condiciones climáticas

   Métrica Principal:
   → Mean Absolute Error (MAE) en segundos

   Métricas Secundarias:
   → RMSE, R², MAPE

   Valor de Negocio:
   → Optimización de estrategia de neumáticos y combustible durante la carrera

   Desafíos Técnicos:
   → Variabilidad por condiciones climáticas, incidentes en pista, safety cars


### 5.2 Caso de Clasificación: Predicción de Podio

In [15]:
# Exploración para caso de clasificación
print("CASO DE USO 2: CLASIFICACIÓN - Predicción de Podio (Top 3)")
print("="*80)

# Análisis de resultados de carrera
if 'race_results' in datasets:
    race_data = datasets['race_results']
    
    print("\n📊 Análisis de Resultados de Carrera:")
    print(f"   - Total de resultados: {len(race_data):,}")
    
    # Análisis de podios si existe la columna Position
    if 'position' in race_data.columns or 'Position' in race_data.columns:
        pos_col = 'Position' if 'Position' in race_data.columns else 'position'
        
        # Calcular estadísticas de podios
        try:
            # Convertir a numérico, manejar valores no numéricos
            race_data['position_numeric'] = pd.to_numeric(race_data[pos_col], errors='coerce')
            
            podium_count = race_data[race_data['position_numeric'] <= 3].shape[0]
            total_finishers = race_data[race_data['position_numeric'].notna()].shape[0]
            
            if total_finishers > 0:
                podium_rate = (podium_count / total_finishers) * 100
                print(f"\n   Estadísticas de Podios:")
                print(f"   - Total de podios: {podium_count:,}")
                print(f"   - Total de finalistas: {total_finishers:,}")
                print(f"   - Tasa de podio: {podium_rate:.2f}%")
        except Exception as e:
            print(f"   - Error calculando estadísticas: {e}")

# Definición del problema de clasificación
classification_problem = pd.DataFrame({
    'Aspecto': [
        'Variable Objetivo',
        'Clases',
        'Variables Predictoras',
        'Métrica Principal',
        'Métricas Secundarias',
        'Valor de Negocio',
        'Desafíos Técnicos'
    ],
    'Descripción': [
        'Finalización en podio (Sí/No)',
        'Binaria: 1 (Podio: posiciones 1-3), 0 (No podio: posición 4+)',
        'Posición de clasificación, rendimiento reciente, histórico en el circuito, fiabilidad del coche',
        'F1-Score (balance entre precisión y recall)',
        'Accuracy, Precision, Recall, AUC-ROC',
        'Apoyo en decisiones de apuestas, estrategias de equipo, contenido mediático',
        'Desbalance de clases (solo ~15% son podios), alta variabilidad por incidentes'
    ]
})

print("\n📋 Definición del Problema de Clasificación:")
for _, row in classification_problem.iterrows():
    print(f"\n   {row['Aspecto']}:")
    print(f"   → {row['Descripción']}")

CASO DE USO 2: CLASIFICACIÓN - Predicción de Podio (Top 3)

📊 Análisis de Resultados de Carrera:
   - Total de resultados: 26,519

   Estadísticas de Podios:
   - Total de podios: 3,360
   - Total de finalistas: 15,591
   - Tasa de podio: 21.55%

📋 Definición del Problema de Clasificación:

   Variable Objetivo:
   → Finalización en podio (Sí/No)

   Clases:
   → Binaria: 1 (Podio: posiciones 1-3), 0 (No podio: posición 4+)

   Variables Predictoras:
   → Posición de clasificación, rendimiento reciente, histórico en el circuito, fiabilidad del coche

   Métrica Principal:
   → F1-Score (balance entre precisión y recall)

   Métricas Secundarias:
   → Accuracy, Precision, Recall, AUC-ROC

   Valor de Negocio:
   → Apoyo en decisiones de apuestas, estrategias de equipo, contenido mediático

   Desafíos Técnicos:
   → Desbalance de clases (solo ~15% son podios), alta variabilidad por incidentes


## 6. Análisis de Calidad de Datos

In [16]:
# Evaluación de calidad de datos
print("EVALUACIÓN DE CALIDAD DE DATOS")
print("="*80)

quality_metrics = []

for name, df in datasets.items():
    total_cells = df.shape[0] * df.shape[1]
    null_cells = df.isnull().sum().sum()
    completeness = ((total_cells - null_cells) / total_cells * 100) if total_cells > 0 else 0
    
    # Verificar duplicados si hay columnas que puedan ser identificadores
    duplicate_rows = 0
    if len(df) > 0:
        duplicate_rows = df.duplicated().sum()
    
    quality_metrics.append({
        'Dataset': name.replace('_', ' ').title(),
        'Filas': f"{len(df):,}",
        'Columnas': df.shape[1],
        'Completitud (%)': f"{completeness:.1f}",
        'Duplicados': duplicate_rows,
        'Calidad': '✓ Alta' if completeness > 95 and duplicate_rows == 0 else 
                   '⚠ Media' if completeness > 80 else '✗ Baja'
    })

quality_df = pd.DataFrame(quality_metrics)
print(quality_df.to_string(index=False))

# Resumen de calidad
high_quality = sum(1 for m in quality_metrics if '✓' in m['Calidad'])
medium_quality = sum(1 for m in quality_metrics if '⚠' in m['Calidad'])
low_quality = sum(1 for m in quality_metrics if '✗' in m['Calidad'])

print(f"\n📊 Resumen de Calidad:")
print(f"   - Datasets de alta calidad: {high_quality}")
print(f"   - Datasets de calidad media: {medium_quality}")
print(f"   - Datasets de baja calidad: {low_quality}")

EVALUACIÓN DE CALIDAD DE DATOS
                Dataset   Filas  Columnas Completitud (%)  Duplicados Calidad
         Driver Details     859         9           100.0           0  ✓ Alta
        Driver Rankings  34,595         7           100.0           0  ✓ Alta
Constructor Performance  12,505         5           100.0           0  ✓ Alta
   Constructor Rankings  13,271         7           100.0           0  ✓ Alta
           Race Results  26,519        19            97.8           0  ✓ Alta
     Qualifying Results  10,254         9           100.0           0  ✓ Alta
            Lap Timings 575,029         6           100.0           0  ✓ Alta
       Pit Stop Records  10,990         7           100.0           0  ✓ Alta
          Race Schedule   1,125        18           100.0           0  ✓ Alta
            Race Status     139         2           100.0           0  ✓ Alta
       Season Summaries      75         2           100.0           0  ✓ Alta
    Sprint Race Results     300  

## 7. Variables Clave Identificadas para Machine Learning

In [17]:
# Identificación de variables clave
print("VARIABLES CLAVE PARA MODELOS DE MACHINE LEARNING")
print("="*80)

# Variables para modelo de regresión
regression_vars = pd.DataFrame({
    'Categoría': [
        'Rendimiento Histórico',
        'Rendimiento Histórico',
        'Rendimiento Histórico',
        'Clasificación',
        'Clasificación',
        'Clasificación',
        'Circuito',
        'Circuito',
        'Constructor',
        'Constructor',
        'Estrategia',
        'Estrategia'
    ],
    'Variable': [
        'avg_lap_time_last_3_races',
        'std_lap_time_last_3_races',
        'best_lap_time_career',
        'qualifying_position',
        'q1_time',
        'q3_time',
        'circuit_length',
        'number_of_turns',
        'constructor_avg_lap_time',
        'constructor_reliability',
        'avg_pit_stop_time',
        'number_of_pit_stops'
    ],
    'Tipo': [
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica'
    ],
    'Importancia Esperada': [
        'Alta',
        'Media',
        'Alta',
        'Alta',
        'Alta',
        'Alta',
        'Media',
        'Media',
        'Alta',
        'Media',
        'Media',
        'Media'
    ]
})

print("\n📊 Variables para Modelo de REGRESIÓN (Tiempo de Vuelta):")
print(regression_vars.to_string(index=False))

# Variables para modelo de clasificación
classification_vars = pd.DataFrame({
    'Categoría': [
        'Posición Inicial',
        'Rendimiento Reciente',
        'Rendimiento Reciente',
        'Rendimiento Reciente',
        'Histórico',
        'Histórico',
        'Constructor',
        'Constructor',
        'Circuito',
        'Experiencia'
    ],
    'Variable': [
        'grid_position',
        'points_last_3_races',
        'avg_position_last_3_races',
        'dnf_last_5_races',
        'career_podium_rate',
        'podiums_at_circuit',
        'constructor_championship_position',
        'teammate_performance',
        'circuit_type',
        'years_in_f1'
    ],
    'Tipo': [
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Numérica',
        'Categórica',
        'Numérica'
    ],
    'Importancia Esperada': [
        'Muy Alta',
        'Alta',
        'Alta',
        'Media',
        'Alta',
        'Media',
        'Alta',
        'Media',
        'Baja',
        'Media'
    ]
})

print("\n📊 Variables para Modelo de CLASIFICACIÓN (Predicción de Podio):")
print(classification_vars.to_string(index=False))

VARIABLES CLAVE PARA MODELOS DE MACHINE LEARNING

📊 Variables para Modelo de REGRESIÓN (Tiempo de Vuelta):
            Categoría                  Variable     Tipo Importancia Esperada
Rendimiento Histórico avg_lap_time_last_3_races Numérica                 Alta
Rendimiento Histórico std_lap_time_last_3_races Numérica                Media
Rendimiento Histórico      best_lap_time_career Numérica                 Alta
        Clasificación       qualifying_position Numérica                 Alta
        Clasificación                   q1_time Numérica                 Alta
        Clasificación                   q3_time Numérica                 Alta
             Circuito            circuit_length Numérica                Media
             Circuito           number_of_turns Numérica                Media
          Constructor  constructor_avg_lap_time Numérica                 Alta
          Constructor   constructor_reliability Numérica                Media
           Estrategia         avg_p

## 8. Plan de Implementación

### 8.1 Arquitectura Propuesta

## 9. Riesgos y Mitigaciones

## 10. Conclusiones y Próximos Pasos

### 10.1 Conclusiones del Análisis

In [21]:
print("CONCLUSIONES DEL ANÁLISIS DE NEGOCIO")
print("="*80)

conclusions = [
    "✓ Los datos disponibles son suficientes para desarrollar modelos predictivos robustos",
    "✓ Existe un claro valor de negocio en ambos casos de uso (regresión y clasificación)",
    "✓ Los stakeholders identificados pueden beneficiarse significativamente de las predicciones",
    "✓ La calidad de datos es generalmente alta, con algunas áreas que requieren preprocesamiento",
    "✓ El plan de implementación es realista y alcanzable en 10 semanas"
]

for conclusion in conclusions:
    print(f"\n{conclusion}")

print("\n" + "="*80)
print("\nPRÓXIMOS PASOS INMEDIATOS:")
print("="*80)

next_steps = [
    "1. Comenzar con la limpieza y preparación de datos",
    "2. Desarrollar el pipeline de feature engineering",
    "3. Implementar modelos baseline para establecer benchmarks",
    "4. Crear visualizaciones exploratorias de las variables clave",
    "5. Documentar todos los supuestos y decisiones técnicas"
]

for step in next_steps:
    print(f"\n   {step}")

CONCLUSIONES DEL ANÁLISIS DE NEGOCIO

✓ Los datos disponibles son suficientes para desarrollar modelos predictivos robustos

✓ Existe un claro valor de negocio en ambos casos de uso (regresión y clasificación)

✓ Los stakeholders identificados pueden beneficiarse significativamente de las predicciones

✓ La calidad de datos es generalmente alta, con algunas áreas que requieren preprocesamiento

✓ El plan de implementación es realista y alcanzable en 10 semanas


PRÓXIMOS PASOS INMEDIATOS:

   1. Comenzar con la limpieza y preparación de datos

   2. Desarrollar el pipeline de feature engineering

   3. Implementar modelos baseline para establecer benchmarks

   4. Crear visualizaciones exploratorias de las variables clave

   5. Documentar todos los supuestos y decisiones técnicas


### 10.2 Resumen Ejecutivo

Este análisis de negocio ha establecido las bases para un proyecto de Machine Learning en Formula 1 con dos casos de uso principales:

1. **Modelo de Regresión**: Predicción de tiempos de vuelta promedio
   - Variable objetivo clara y medible
   - Aplicación directa en estrategia de carrera
   - Métricas de éxito bien definidas (MAE < 1.5 segundos)

2. **Modelo de Clasificación**: Predicción de podios
   - Alto valor para múltiples stakeholders
   - Desafío técnico interesante por desbalance de clases
   - Potencial de monetización significativo

El proyecto está listo para avanzar a la fase de implementación técnica con un roadmap claro y métricas de éxito definidas.